In [13]:
import pandas as pd
from collections import Counter
from datetime import datetime
from sklearn.decomposition import PCA

In [14]:
total_df = pd.read_csv("total_df.csv")

In [3]:
# Print counts of each data type to check that they are correct
print("COLUMN NAME --- EXAMPLE --- DATA TYPES --- NUMEBR OF DIFFERENT VALUES")
for col in total_df.columns:
    types = [type(total_df[col][i]) for i in range(len(total_df))]
    print(col,"---", total_df[col][0], "---", Counter(types), " --- ", len(Counter(total_df[col])))

COLUMN NAME --- EXAMPLE --- DATA TYPES --- NUMEBR OF DIFFERENT VALUES
countryName --- Germany --- Counter({<class 'str'>: 65628})  ---  32
eprtrSectorName --- Mineral industry --- Counter({<class 'str'>: 65628})  ---  9
EPRTRAnnexIMainActivityLabel --- Installations for the production of cement clinker in rotary kilns --- Counter({<class 'str'>: 65628})  ---  71
FacilityInspireID --- https://registry.gdi-de.org/id/de.ni.mu/06221720040 --- Counter({<class 'str'>: 65628})  ---  7185
facilityName --- Holcim (Deutschland) GmbH Werk Höver --- Counter({<class 'str'>: 65628})  ---  7930
City --- Sehnde --- Counter({<class 'str'>: 65628})  ---  5136
targetRelease --- AIR --- Counter({<class 'str'>: 65628})  ---  1
pollutant --- Carbon dioxide (CO2) --- Counter({<class 'str'>: 65628})  ---  3
reportingYear --- 2015 --- Counter({<class 'numpy.int64'>: 65628})  ---  14
MONTH --- 10 --- Counter({<class 'numpy.int64'>: 65628})  ---  12
DAY --- 20 --- Counter({<class 'numpy.int64'>: 65628})  ---  28

In [4]:
total_df[total_df["facilityName"] == "Sidec"][["MONTH","DAY", "reportingYear"]]

,MONTH,DAY,reportingYear
9462,3,28,2010
9644,5,18,2019
14646,7,17,2017
14690,4,19,2013
19881,5,3,2011
23417,12,27,2012
29102,12,27,2012
32280,3,28,2010
32722,3,28,2010
32989,3,28,2010


In [15]:
# Eliminate variables that will not help with the classification, as they are redundant or do not provide valuable info
# In particular, delete "FacilityInspireID", "targetRelease", "REPORTER NAME", "CITY ID", "CONTINENT"
total_df.drop(columns = ["FacilityInspireID", "targetRelease", "REPORTER NAME", "CITY ID", "CONTINENT"], inplace = True) # Drop redundant columns

In [16]:
# Time to numeric between min and max
# To do that, convert month and day into day of the year (from 1 to 365), and leave the year as it is.
# The intuition of this is that the wind and temperature is related to the day of the year and that a factory
# operating at a determined year could also be related to the pollutant.
dates = total_df[["MONTH", "DAY", "reportingYear"]]
day_of_year = [int(datetime(dates.loc[i, "reportingYear"], dates.loc[i, "MONTH"], dates.loc[i, "DAY"]).strftime('%j')) for i in range(len(dates))]
print(dates.iloc[0])
print(day_of_year[0], len(day_of_year))
total_df["DAY"] = day_of_year
total_df.drop(columns = "MONTH", inplace = True)
total_df.head()

MONTH              10
DAY                20
reportingYear    2015
Name: 0, dtype: int64
293 65628


,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,facilityName,City,pollutant,reportingYear,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS
0,Germany,Mineral industry,Installations for the production of cement cli...,Holcim (Deutschland) GmbH Werk Höver,Sehnde,Carbon dioxide (CO2),2015,293,15.118767,14.312541,21.419106,2.864895,4.924169,9.688206,2
1,Italy,Mineral industry,Installations for the production of cement cli...,Stabilimento di Tavernola Bergamasca,TAVERNOLA BERGAMASCA,Nitrogen oxides (NOX),2018,264,19.661550,19.368166,21.756389,5.462839,7.864403,12.023521,1
2,Spain,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,COMPLEJO MEDIOAMBIENTAL DE ZURITA,PUERTO DEL ROSARIO,Methane (CH4),2019,35,12.729453,14.701985,17.103930,1.511201,4.233438,8.632193,2
3,Czechia,Energy sector,Thermal power stations and other combustion in...,Elektrárny Prunéřov,Kadaň,Nitrogen oxides (NOX),2012,219,11.856417,16.122584,17.537184,10.970301,10.298348,15.179215,0
4,Finland,Waste and wastewater management,Urban waste-water treatment plants,"TAMPEREEN VESI LIIKELAITOS, VIINIKANLAHDEN JÄT...",Tampere,Methane (CH4),2018,356,17.111930,20.201604,21.536012,11.772039,11.344078,16.039004,2


In [17]:
# Scale the numerical variables to the [0, 1] range
variables_to_scale = ["DAY", "reportingYear", "max_wind_speed", "avg_wind_speed", "min_wind_speed", "max_temp", "avg_temp", "min_temp", "DAY WITH FOGS"]
for var in variables_to_scale:
    col = total_df[var]
    a = (col-col.min()).div(col.max()-col.min())
    total_df[var] = (col-col.min()).div(col.max()-col.min())

total_df.head()

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,facilityName,City,pollutant,reportingYear,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS
0,Germany,Mineral industry,Installations for the production of cement cli...,Holcim (Deutschland) GmbH Werk Höver,Sehnde,Carbon dioxide (CO2),0.615385,0.806630,0.474438,0.039056,0.428596,0.249436,0.253649,0.366280,0.105263
1,Italy,Mineral industry,Installations for the production of cement cli...,Stabilimento di Tavernola Bergamasca,TAVERNOLA BERGAMASCA,Nitrogen oxides (NOX),0.846154,0.726519,0.777706,0.671019,0.451231,0.357326,0.399215,0.463555,0.052632
2,Spain,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,COMPLEJO MEDIOAMBIENTAL DE ZURITA,PUERTO DEL ROSARIO,Methane (CH4),0.923077,0.093923,0.314932,0.087737,0.139007,0.193219,0.219452,0.322292,0.105263
3,Czechia,Energy sector,Thermal power stations and other combustion in...,Elektrárny Prunéřov,Kadaň,Nitrogen oxides (NOX),0.384615,0.602210,0.256649,0.265315,0.168082,0.586043,0.519716,0.595002,0.000000
4,Finland,Waste and wastewater management,Urban waste-water treatment plants,"TAMPEREEN VESI LIIKELAITOS, VIINIKANLAHDEN JÄT...",Tampere,Methane (CH4),0.846154,0.980663,0.607498,0.775200,0.436442,0.619338,0.571488,0.630816,0.105263


In [ ]:
# One hot encode the categorical variables

# Load test dataset to consider all possible values
#test_df = pd.read_csv("data/test_x.csv")
#variables_to_encode = ["countryName", "eprtrSectorName", "EPRTRAnnexIMainActivityLabel", "facilityName", "City"]
#for var in variables_to_encode:
#    possibilities = list(test_df[var])
#    possibilities.extend(list(total_df[var]))
#    possibilities = list(Counter(possibilities).keys())
#    print("Variable:", var, "  Different values:", len(possibilities))
#    encoded_cols = pd.get_dummies(total_df[var], prefix=var)
#    # Append the one-hot encoded columns
#    total_df = total_df.join(encoded_cols)
    

# Delete the original variables
#total_df.drop(columns = variables_to_encode, inplace = True)
#print("Resulting shape", total_df.shape)

# Save training dataset
total_df.to_csv("clean_train.csv", index = False)

In [11]:
# One hot encode the categorical variables, with dimensionality reduction

# Load test dataset to consider all possible values
test_df = pd.read_csv("data/test_x.csv")
variables_to_encode = ["countryName", "eprtrSectorName", "EPRTRAnnexIMainActivityLabel", "facilityName", "City"]
for var in variables_to_encode:
    possibilities = list(test_df[var])
    possibilities.extend(list(total_df[var]))
    possibilities = list(Counter(possibilities).keys())
    print("Variable:", var, "  Different values:", len(possibilities))
    encoded_cols = pd.get_dummies(total_df[var], prefix=var)
    if var ==  "facilityName" or var == "City":
        pca = PCA(n_components=10)
        principalComponents = pca.fit_transform(encoded_cols)
        principalDf = pd.DataFrame(data = principalComponents
                , columns = [var+ 'PC' + str(i) for i in range(10)])
    else:
        pca = PCA(n_components=3)
        principalComponents = pca.fit_transform(encoded_cols)
        principalDf = pd.DataFrame(data = principalComponents
                , columns = [var+ 'PC' + str(i) for i in range(3)])
    # Append the reduced columns
    total_df = total_df.join(principalDf)

# Delete the original variables
total_df.drop(columns = variables_to_encode, inplace = True)
print("Resulting shape", total_df.shape)
total_df.to_csv("clean_train_reduced.csv", index = False) # Save reduced dataset

Variable: countryName   Different values: 32
Variable: eprtrSectorName   Different values: 9
Variable: EPRTRAnnexIMainActivityLabel   Different values: 71
Variable: facilityName   Different values: 8516
Variable: City   Different values: 5426
Resulting shape (65628, 39)


In [19]:
total_df.head()

,pollutant,reportingYear,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,...,facilityNamePC1,facilityNamePC2,facilityNamePC3,facilityNamePC4,facilityNamePC5,CityPC1,CityPC2,CityPC3,CityPC4,CityPC5
0,Carbon dioxide (CO2),0.615385,0.806630,0.474438,0.039056,0.428596,0.249436,0.253649,0.366280,0.105263,...,-0.003593,-0.001864,-0.001664,-0.001782,-0.001660,-0.029846,-0.005010,-0.004080,-0.002884,-0.003514
1,Nitrogen oxides (NOX),0.846154,0.726519,0.777706,0.671019,0.451231,0.357326,0.399215,0.463555,0.052632,...,-0.003560,-0.001828,-0.001633,-0.001743,-0.001626,-0.029814,-0.004978,-0.004048,-0.002855,-0.003478
2,Methane (CH4),0.923077,0.093923,0.314932,0.087737,0.139007,0.193219,0.219452,0.322292,0.105263,...,-0.003402,-0.001670,-0.001476,-0.001571,-0.001455,-0.030004,-0.005171,-0.004247,-0.003039,-0.003701
3,Nitrogen oxides (NOX),0.384615,0.602210,0.256649,0.265315,0.168082,0.586043,0.519716,0.595002,0.000000,...,-0.003714,-0.001985,-0.001804,-0.001931,-0.001822,-0.030244,-0.005434,-0.004523,-0.003316,-0.004022
4,Methane (CH4),0.846154,0.980663,0.607498,0.775200,0.436442,0.619338,0.571488,0.630816,0.105263,...,-0.003387,-0.001656,-0.001462,-0.001556,-0.001440,-0.030554,-0.005803,-0.004923,-0.003712,-0.004592


In [20]:
# Generate the reduced testing dataset
test_df = pd.read_csv("data/test_x.csv")
test_df.drop(columns = ["FacilityInspireID", "targetRelease", "REPORTER NAME", "CITY ID", "CONTINENT", "EPRTRAnnexIMainActivityCode", "EPRTRSectorCode"], inplace = True) # Drop redundant columns

dates = test_df[["MONTH", "DAY", "reportingYear"]]
day_of_year = [int(datetime(dates.loc[i, "reportingYear"], dates.loc[i, "MONTH"], dates.loc[i, "DAY"]).strftime('%j')) for i in range(len(dates))]
print(dates.iloc[0])
print(day_of_year[0], len(day_of_year))
test_df["DAY"] = day_of_year
test_df.drop(columns = "MONTH", inplace = True)
test_df.head()

variables_to_encode = ["countryName", "eprtrSectorName", "EPRTRAnnexIMainActivityLabel", "facilityName", "City"]
for var in variables_to_encode:
    possibilities = list(test_df[var])
    possibilities.extend(list(total_df[var]))
    possibilities = list(Counter(possibilities).keys())
    print("Variable:", var, "  Different values:", len(possibilities))
    encoded_cols = pd.get_dummies(test_df[var], prefix=var)
    if var ==  "facilityName" or var == "City":
        pca = PCA(n_components=10)
        principalComponents = pca.fit_transform(encoded_cols)
        principalDf = pd.DataFrame(data = principalComponents
                , columns = [var+ 'PC' + str(i) for i in range(10)])
    else:
        pca = PCA(n_components=3)
        principalComponents = pca.fit_transform(encoded_cols)
        principalDf = pd.DataFrame(data = principalComponents
                , columns = [var+ 'PC' + str(i) for i in range(3)])
    # Append the reduced columns
    test_df = test_df.join(principalDf)

# Delete the original variables
test_df.drop(columns = variables_to_encode, inplace = True)
print("Resulting shape", test_df.shape)
test_df.to_csv("clean_test_reduced.csv", index = False) # Save reduced dataset

MONTH               8
DAY                16
reportingYear    2017
Name: 0, dtype: int64
228 24480
Variable: countryName   Different values: 32
Variable: eprtrSectorName   Different values: 9
Variable: EPRTRAnnexIMainActivityLabel   Different values: 71
Variable: facilityName   Different values: 8516
Variable: City   Different values: 5426
Resulting shape (24480, 39)
